In [1]:
from joblib import load
import csv
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

C:\Users\12406\AppData\Local\Temp\ipykernel_10432\1219203286.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dt_classifier = load('dt_classifier.joblib')

In [3]:
with open('NBA Champion RawData.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    data = list(csv_reader)

In [4]:
df = pd.DataFrame(data[1:], columns=data[0])

In [5]:
df_2023 = df[df['\ufeffSeason'] == '2023']

In [6]:
feature_names = ['rk conference', 'sum mvp shares', 'Conference OR', 'overall record', 'MOV']
feature_df_2023 = df_2023[feature_names]
feature_df_2023.head()

,rk conference,sum mvp shares,Conference OR,overall record,MOV
30,1,2.907,0.515679443,0.707317073,3.63
31,2,0.397,0.515679443,0.695121951,6.52
32,3,4.994,0.515679443,0.658536585,4.32
33,1,2.064,0.504878049,0.646341463,3.33
34,2,0.01,0.504878049,0.62195122,3.94


In [7]:
label_names = ['champion']
label_df_2023 = df_2023[label_names]
label_df_2023.head()

,champion
30,0
31,0
32,0
33,1
34,0


In [8]:
y_scores = dt_classifier.predict_proba(feature_df_2023)[:, 1]
y_scores

array([0.94892168, 0.74829932, 0.94892168, 0.94892168, 0.21052632,
       0.        , 0.21052632, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [9]:
training_predictions = dt_classifier.predict(feature_df_2023)
training_accuracy = accuracy_score(label_df_2023, training_predictions)
print("Training Accuracy:", training_accuracy)

training_conf_matrix = confusion_matrix(label_df_2023, training_predictions)
tn_train, fp_train, fn_train, tp_train = training_conf_matrix.ravel()
false_negative_rate_train = fn_train / (fn_train + tp_train)
print("Training False Negative Rate:", false_negative_rate_train)


false_positive_rate_train = fp_train / (fp_train + tn_train)
print("Training False Positive Rate:", false_positive_rate_train)


Training Accuracy: 0.9
Training False Negative Rate: 0.0
Training False Positive Rate: 0.10344827586206896
